In [14]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [15]:
train_set=pd.read_csv('new_train.csv')
test_set=pd.read_csv('new_test.csv')

In [16]:
cols=['first_active_month','card_id','feature_1','feature_2','feature_3','num_of_transactions','hist_sum','hist_mean','hist_std'
     ,'hist_min','hist_max','num_mer_transactions','mer_sum','mer_mean','mer_min','mer_max']
target_col='target'

In [17]:
xtrain=train_set[cols]
ytrain=train_set[target_col]
xtest=test_set[cols]

In [18]:
#convert first active month to date, month and year respectively
mnth=pd.DatetimeIndex(xtrain['first_active_month']).month
#day=xtrain['first_active_month'].dt.day
year=pd.DatetimeIndex(xtrain['first_active_month']).year

#xtrain['day']=day
xtrain['month']=mnth
xtrain['year']=year

xtrain=xtrain.drop(['first_active_month','card_id'],axis=1)

xtrain.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,feature_1,feature_2,feature_3,num_of_transactions,hist_sum,hist_mean,hist_std,hist_min,hist_max,num_mer_transactions,mer_sum,mer_mean,mer_min,mer_max,month,year
0,5,2,1,260,-165.968739,-0.638341,0.212139,-0.739395,2.258395,23.0,-13.244202,-0.575835,-0.724368,-0.296112,6,2017
1,4,1,0,350,-210.006336,-0.600018,0.384967,-0.742400,4.630299,6.0,-4.355735,-0.725956,-0.739410,-0.701858,1,2017
2,2,2,0,43,-29.167391,-0.678311,0.087380,-0.730138,-0.145847,1.0,-0.700326,-0.700326,-0.700326,-0.700326,8,2016


In [19]:
#for test data
mnth=pd.DatetimeIndex(xtest['first_active_month']).month
year=pd.DatetimeIndex(xtest['first_active_month']).year

xtest['month']=mnth
xtest['year']=year

#drop the first_active_month column
xtest=xtest.drop(['first_active_month','card_id'],axis=1)
xtest.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,feature_1,feature_2,feature_3,num_of_transactions,hist_sum,hist_mean,hist_std,hist_min,hist_max,num_mer_transactions,mer_sum,mer_mean,mer_min,mer_max,month,year
0,3,3,1,68,-40.733733,-0.599025,0.192268,-0.743902,0.235676,3.0,-1.777156,-0.592385,-0.722114,-0.383266,4.0,2017.0
1,2,3,0,78,-49.136513,-0.629955,0.154999,-0.731881,0.318818,9.0,-5.944698,-0.660522,-0.740897,-0.506484,1.0,2017.0
2,5,1,1,13,4.528840,0.348372,0.906547,-0.536537,2.525866,2.0,0.180138,0.090069,-0.724368,0.904506,8.0,2017.0


In [20]:
elo_dmatrix=lgb.Dataset(xtrain,ytrain)

In [76]:
param_grid={
"num_leaves":[30],
'min_child_weight': [50],
'learning_rate': [0.05],
'bagging_fraction': [0.7],
'feature_fraction': [0.7],
'bagging_frequency':[5],
'bagging_seed':[102],
'verbosity':[-1]}

In [77]:
#instantiate lightgbm model
lgb_model=lgb.LGBMRegressor(objective='regression',metric='rmse',num_boost_round=1000)

In [78]:
#instantiate grid search 
grid_mse=GridSearchCV(param_grid=param_grid,estimator=lgb_model,scoring='neg_mean_squared_error',cv=4,verbose=1)

In [79]:
grid_mse.fit(xtrain,ytrain)

Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   29.5s finished
/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn(

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       metric='rmse', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=100, n_jobs=-1,
       num_boost_round=1000, num_leaves=31, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'num_leaves': [30], 'min_child_weight': [50], 'learning_rate': [0.05], 'bagging_fraction': [0.7], 'feature_fraction': [0.7], 'bagging_frequency': [5], 'bagging_seed': [102], 'verbosity': [-1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [80]:
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Best parameters found:  {'bagging_fraction': 0.7, 'bagging_frequency': 5, 'bagging_seed': 102, 'feature_fraction': 0.7, 'learning_rate': 0.05, 'min_child_weight': 50, 'num_leaves': 30, 'verbosity': -1}
Lowest RMSE found:  3.8173219574505426


In [81]:
preds=grid_mse.predict(xtest)

In [82]:
submission_df=pd.DataFrame()
submission_df['card_id']=test_set['card_id']

In [83]:
submission_df['target']=preds

In [84]:
submission_df.to_csv('lgb_submission.csv',index=False)